# Planning-Lab Lesson 2: Informed Search Strategies

In the second session we will work on informed search

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

### Priority Queue

You will need a queue ordered by priority as a queue or **PriorityQueue**. The difference between the other versions of the queue is that in **PriorityQueue**, nodes are removed from the data structure based on the current lowest value. In particular, **Node** has two useful parameters (other than those used in the previous session):

- pathcost - the path cost from the root node to the current one (defaults to 0)
- value - the value of a node. Used by PriorityQueue to order its content (defaults to 0)

### Here is an example of usage:

In [16]:
import os
import sys
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *

# Create 3 nodes for state ids 1 2 3
node_1 = Node(1) # No parent, pathcost=0, value=0
node_2 = Node(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, pathcost=1, value=10
node_3 = Node(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_queue = PriorityQueue()
for n in (node_1, node_2, node_3):
    p_queue.add(n)
    print("Added: {}".format(n.state))

while not p_queue.is_empty():
    node = p_queue.remove()
    print("Removed state : {}, path cost: {}, value: {}".format(node.state,node.pathcost,node.value))

Added: 1
Added: 2
Added: 3
Removed state : 1, path cost: 0, value: 0
Removed state : 3, path cost: 100, value: 5
Removed state : 2, path cost: 1, value: 10


Notice the order in which nodes are removed from the queue.

## Uniform-Cost Search (UCS)
Informed strategy can be considered as a specific implementation of Uniform-Cost Search (UCS). The following code implements a UCS, *graph search* version. The cost of performing an action is supposed to be always 1 (also in the assignments).

In [17]:
def present_with_higher_cost(queue, node):
    if node.state in queue:
        if queue[node.state].pathcost > node.pathcost: 
            return True
    return False

In [18]:
import gym
import envs

def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """
    
    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    
    explored = set()
    time_cost = 1
    space_cost = 1
    
    while True:
        if queue.is_empty(): 
            return None, time_cost, space_cost 
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(environment.sample(node.state, action), node, node.pathcost + 1, node.pathcost + 1)  
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                
            elif present_with_higher_cost(queue, child):
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))


#### Let's see the results:

In [19]:
# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution, time, memory = ucs(env)

CheckResult_UCS(solution, time, memory, env)

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

##########################################
#####  UNIFORM GRAPH SEARCH PROBLEM  #####
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 61
Max n° of nodes in memory: 16


## Distance Heuristics

Informed search requires a distance heuristic to estimate the distance between a state and the goal. You already have at your disposal these functions:

- *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates.
- *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates.
- *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to the L1 norm but diagonal moves are also considered.


**Examples:**

In [20]:
p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(Heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(Heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(Heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


# Assignment 1: Greedy Best-First Search

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you have to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use the L1 norm as a heuristic function first, then try the others to see the differences.

The results returned by greedy must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.


### Functions to implement:
- *greedy_tree_search(environment)*
- *greedy_graph_search(environment)*

**The following function is a revised version of the present_with_higher_cost function that now checks for the value of the node, not the path cost. This is the version you should use in the graph search implementation of greedy and A***

In [21]:
def present_with_higher_value(queue, node):
    if node.state in queue:
        if queue[node.state].value > node.value: 
            return True
    return False

In [22]:
def greedy_tree_search(environment, heuristic_function, timeout=10000):
    """
    Greedy-best-first Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    goalpos = environment.state_to_pos(environment.goalstate)
    
    time_cost = 1
    space_cost = 1

    while True:
        if time_cost >= timeout: return ("time-out", time_cost, space_cost) # timeout check
        if queue.is_empty(): 
            return None, time_cost, space_cost 
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
                
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(
                environment.sample(node.state, action),
                node, node.pathcost + 1,
                heuristic_function(
                    environment.state_to_pos(environment.sample(node.state, action)),
                    goalpos
                )
            )  
            time_cost += 1
            
            queue.add(child)
                
        space_cost = max(space_cost, len(queue))

In [23]:
def greedy_graph_search(environment, heuristic_function):
    """
    Greedy-best-first Graph search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    goalpos = environment.state_to_pos(environment.goalstate)
    
    explored = set()
    time_cost = 1
    space_cost = 1
    while True:
        if queue.is_empty(): 
            return None, time_cost, space_cost 
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(
                environment.sample(node.state, action),
                node, node.pathcost + 1,
                heuristic_function(
                    environment.state_to_pos(environment.sample(node.state, action)),
                    goalpos
                )
            )  
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                
            elif present_with_higher_value(queue, child):
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))

**The following function calls your implementations of greedy_tree_search and greedy_graph_search:**

In [24]:
def greedy(environment, search_type, heuristic_function):
    """
    Greedy-best-first search
    
    Args:
        problem: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    path, time_cost, space_cost = search_type(environment, heuristic_function)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of Greedy-best-first search and checks the results:**

In [25]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.l1_norm
 
solution_ts, time_ts, memory_ts = greedy(environment, greedy_tree_search, heuristic_function)
solution_gs, time_gs, memory_gs = greedy(environment, greedy_graph_search, heuristic_function)

heuristic = ""
if heuristic_function == Heu.l1_norm:
    heuristic = "l1_norm"
elif heuristic_function == Heu.l2_norm:
    heuristic = "l2_norm"
elif heuristic_function == Heu.chebyshev:
    heuristic = "chebyshev"
    
results = CheckResult_L2A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

########################################################
#######  GREEDY BEST FIRST TREE SEARCH PROBLEM  ########
########################################################
Your solution: time-out
N° of nodes explored: 10001
Max n° of nodes in memory: 7501

===> Your solution is correct!

########################################################
#######  GREEDY BEST FIRST GRAPH SEARCH PROBLEM  #######
########################################################
Your solution: [(0, 3), (1, 3), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 45
Max n° of nodes in memory: 15

===> Your solution is correct!



# Assignment 2: A* Search
The second assignment is to implement the A* search algorithm on SmallMaze. In particular, you have to implement both astar_tree_search and astar_graph_search versions that the generic astar will call. Use the L1 norm as a heuristic function first, then try the others to see the differences.

The results returned by astar must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

Functions to implement:
- *astar_tree_search(environment)*
- *astar_graph_search(environment)*

In [26]:
def astar_tree_search(environment, heuristic_function):
    """
    A* Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    goalpos = environment.state_to_pos(environment.goalstate)
    
    time_cost = 1
    space_cost = 1

    while True:
        if queue.is_empty(): 
            return None, time_cost, space_cost 
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
                
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(
                environment.sample(node.state, action),
                node, node.pathcost + 1,
                (node.pathcost + 1) + heuristic_function(
                    environment.state_to_pos(environment.sample(node.state, action)),
                    goalpos
                )
            )  
            time_cost += 1
            
            queue.add(child)
                
        space_cost = max(space_cost, len(queue))

In [27]:
def astar_graph_search(environment, heuristic_function):
    """
    A* Graph Search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    goalpos = environment.state_to_pos(environment.goalstate)
    
    explored = set()
    time_cost = 1
    space_cost = 1
    while True:
        if queue.is_empty(): 
            return None, time_cost, space_cost 
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(
                environment.sample(node.state, action),
                node, node.pathcost + 1,
                (node.pathcost + 1) + heuristic_function(
                    environment.state_to_pos(environment.sample(node.state, action)),
                    goalpos
                )
            )  
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                
            elif present_with_higher_value(queue, child):
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))

**The following function calls your implementations of astar_tree_search and astar_graph_search:**

In [28]:
def astar(environment, search_type, heuristic_function):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment, heuristic_function)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of A* search and checks the results:**

In [29]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.l1_norm

solution_ts, time_ts, memory_ts = astar(environment, astar_tree_search, heuristic_function)
solution_gs, time_gs, memory_gs = astar(environment, astar_graph_search, heuristic_function)

heuristic = ""
if heuristic_function == Heu.l1_norm:
    heuristic = "l1_norm"
elif heuristic_function == Heu.l2_norm:
    heuristic = "l2_norm"
elif heuristic_function == Heu.chebyshev:
    heuristic = "chebyshev"

results = CheckResult_L2A2([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

#########################################
#######  A* TREE SEARCH PROBLEM  ########
#########################################
Your solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 8361
Max n° of nodes in memory: 6271

===> Your solution is correct!

##########################################
#######  A* GRAPH SEARCH PROBLEM  ########
##########################################
Your solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 61
Max n° of nodes in memory: 16

===> Your solution is correct!



### Discussion
Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.

# Quiz Informed Search
## Question 1
Consider the following environment called GrdMaze-v0:
```python
[
  ['C', 'C', 'C', 'S'],
  ['C', 'C', 'W', 'C'],
  ['C', 'C', 'C', 'C'],
  ['C', 'W', 'W', 'W'],
  ['C', 'C', 'C', 'G'],
]
startstate = 3
goalstate = 19

# You can create this environment with the following commands
envname = "GrdMaze-v0"
environment = gym.make(envname)
```
### Which one of the following answers is correct?
- [ ] a. A* tree search with *l1_norm* explores fewer nodes than A* graph search with *l2_norm*
- [ ] b. A* tree search with *l1_norm* explores 55145 nodes, i.e., the same number of nodes as A* tree search with *l2_norm*
- [x] c. A* tree searcfh with *l1_norm* explores 55145 nodes while A* graph search with *l2_norm* 61.

In [35]:
envname = "GrdMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.l1_norm
_, l1_time_ts, l1_memory_ts = astar(environment, astar_tree_search, heuristic_function)
_, l1_time_gs, l1_memory_gs = astar(environment, astar_graph_search, heuristic_function)

heuristic_function = Heu.l2_norm
_, l2_time_ts, l2_memory_ts = astar(environment, astar_tree_search, heuristic_function)
_, l2_time_gs, l2_memory_gs = astar(environment, astar_graph_search, heuristic_function)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 1                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print("Tree search:")
print("\tNodes explored with l1_norm", l1_time_ts)
print("\tNodes explored with l2_norm", l2_time_ts)

print("\nGraph search:")
print("\tNodes explored with l1_norm", l1_time_gs)
print("\tNodes explored with l2_norm", l2_time_gs)

########################################################
####### QUESTION 1                               #######
########################################################
Tree search:
	Nodes explored with l1_norm 55145
	Nodes explored with l2_norm 56173
Graph search:
	Nodes explored with l1_norm 61
	Nodes explored with l2_norm 61


## Question 2
Consider the following environment called GrdMaze-v0:
```python
[
  ['C', 'C', 'C', 'S'],
  ['C', 'C', 'W', 'C'],
  ['C', 'C', 'C', 'C'],
  ['C', 'W', 'W', 'W'],
  ['C', 'C', 'C', 'G'],
]
startstate = 3
goalstate = 19

# You can create this environment with the following commands
envname = "GrdMaze-v0"
environment = gym.make(envname)
```
### Which one of the following answers is correct?
- [ ] a. Greedy Best First Graph Search with *Chebyshev* heuristic esplores 45 nodes, while with *l1_norm* 49
- [x] b. Greedy Best First Tree Search with *l1_norm* reaches the time-out set to 10000, while Greedy Best Firse Graph Search with the same heuristic returns the solution exploring 45 nodes.
- [ ] c. Greedy Best First Tree Search with *l1_norm* explores 7501 nodes

In [43]:
envname = "GrdMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.l1_norm
l1_sol_ts, l1_time_ts, l1_memory_ts = greedy(environment, greedy_tree_search, heuristic_function)
l1_sol_gs, l1_time_gs, l1_memory_gs = greedy(environment, greedy_graph_search, heuristic_function)

heuristic_function = Heu.chebyshev
ch_sol_ts, ch_time_ts, ch_memory_ts = greedy(environment, greedy_tree_search, heuristic_function)
ch_sol_gs, ch_time_gs, ch_memory_gs = greedy(environment, greedy_graph_search, heuristic_function)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 2                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print("Tree search:")
print("\tSolution with l1_norm", l1_sol_ts)
print("\tNodes explored with l1_norm", l1_time_ts)
print()

print("\tSolution with Chebyshev", ch_sol_ts)
print("\tNodes explored with Chebyshev", ch_time_ts)


print("\nGraph search:")
print("\tSolution with l1_norm", l1_sol_gs)
print("\tNodes explored with l1_norm", l1_time_gs)
print()

print("\tSolution with Chebyshev", ch_sol_gs)
print("\tNodes explored with Chebyshev", ch_time_gs)

########################################################
####### QUESTION 2                               #######
########################################################
Tree search:
	Solution with l1_norm time-out
	Nodes explored with l1_norm 10001

	Solution with Chebyshev time-out
	Nodes explored with Chebyshev 10001

Graph search:
	Solution with l1_norm (7, 11, 10, 9, 8, 12, 16, 17, 18, 19)
	Nodes explored with l1_norm 45

	Solution with Chebyshev (7, 11, 10, 9, 8, 12, 16, 17, 18, 19)
	Nodes explored with Chebyshev 49


## Question 3
Consider the following environment called GrdMaze-v0:
```python
[
  ['C', 'C', 'C', 'S'],
  ['C', 'C', 'W', 'C'],
  ['C', 'C', 'C', 'C'],
  ['C', 'W', 'W', 'W'],
  ['C', 'C', 'C', 'G'],
]
startstate = 3
goalstate = 19

# You can create this environment with the following commands
envname = "GrdMaze-v0"
environment = gym.make(envname)
```
### Which one of the following answers is correct?
- [ ] a. A* Graph Search with *Chebyshev* heuristic finds the solution in this environment but with higher memory usage than UCS Graph Search
- [ ] b. A* Tree Search with *l2_norm* heuristic explores 41359 to find the solution
- [x] c. UCS explores 61 nodes to find the solution

In [47]:
envname = "GrdMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.chebyshev
ch_sol_ts, ch_time_ts, ch_memory_ts = astar(environment, astar_tree_search, heuristic_function)
ch_sol_gs, ch_time_gs, ch_memory_gs = astar(environment, astar_graph_search, heuristic_function)

heuristic_function = Heu.l2_norm
l2_sol_ts, l2_time_ts, l2_memory_ts = astar(environment, astar_tree_search, heuristic_function)

ucs_sol, ucs_time, ucs_memory = ucs(env)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 3                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print("Tree search:")
print("\tNodes explored with l2_norm", l2_time_ts)
print()

print("\nGraph search:")
print("\tNodes explored with UCS", ucs_time)
print("\tNodes memorized with UCS", ucs_memory)
print("\tNodes memorized with Chebyshev", ch_memory_gs)

########################################################
####### QUESTION 3                               #######
########################################################
Tree search:
	Nodes explored with l2_norm 56173


Graph search:
	Nodes explored with UCS 61
	Nodes memorized with UCS 16
	Nodes memorized with Chebyshev 16


## Question 4
Consider the environment called SmallMaze-v0:
```python
# You can create this environment with the following commands
envname = "SmallMaze-v0"
environment = gym.make(envname)
```
### Which one of the following answers is correct?
- [x] a. A* Graph Search with *l2_norm* finds the solution with a total memory usage of 16 nodes
- [ ] b. A* Tree Search with *l1_norm* finds the **optimal solution** with a memory usage of 6500
- [ ] c. UCS explores more nodes to find the solution with regards to A* Graph Search with *l2_norm* which explores 61 nodes

In [57]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.chebyshev
ch_sol_ts, ch_time_ts, ch_memory_ts = astar(environment, astar_tree_search, heuristic_function)
ch_sol_gs, ch_time_gs, ch_memory_gs = astar(environment, astar_graph_search, heuristic_function)

heuristic_function = Heu.l2_norm
l2_sol_gs, l2_time_gs, l2_memory_gs = astar(environment, astar_graph_search, heuristic_function)

heuristic_function = Heu.l1_norm
l1_sol_ts, l1_time_ts, l1_memory_ts = astar(environment, astar_tree_search, heuristic_function)


ucs_sol, ucs_time, ucs_memory = ucs(env)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 4                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

is_optimal = False
if solution_2_string(l1_sol_gs, environment) == [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]:
    is_optimal = True
    
print("Tree search:")
print("\tSolution with A* l1_norm", l1_sol_gs)
if is_optimal:
    print("\t\tThe solution is optimal")
else:
    print("\t\tThe solution is NOT optimal")
print("\tNodes explored with A* l1_norm", l1_time_ts)
print("\tNodes memorized with A* l1_norm", ucs_memory)

print()

print("\nGraph search:")
print("\tNodes explored with A* l2_norm", l2_time_gs)
print("\tNodes memorized with A* l2_norm", l2_memory_gs)
print("\tNodes explored with UCS", ucs_time)

########################################################
####### QUESTION 4                               #######
########################################################
Tree search:
	Solution with A* l1_norm (7, 11, 10, 9, 8, 12, 16, 17, 18, 19)
		The solution is NOT optimal
	Nodes explored with A* l1_norm 8361
	Nodes memorized with A* l1_norm 16


Graph search:
	Nodes explored with A* l2_norm 61
	Nodes memorized with A* l2_norm 16
	Nodes explored with UCS 61


## Question 5
Consider the environment called BlockedMaze-v0:
```python
[
  ['C', 'C', 'S', 'C'],
  ['C', 'C', 'W', 'C'],
  ['C', 'C', 'C', 'C'],
  ['C', 'C', 'W', 'W'],
  ['C', 'C', 'W', 'G'],
]
startstate = 2
goalstate = 19

# You can create this environment with the following commands
envname = "BlockedMaze-v0"
environment = gym.make(envname)
```
### Which one of the following answers is correct?
- [x] a. A* Graph Search with *l1_norm* returns correctly **None** as solution, exploring 61 nodes
- [ ] b. Greedy Best First Tree Search returns correctly **None** as solution, exploring 987 nodes
- [ ] c. A* Graph Search with *l1_norm* returns correctly **None** as solution, exploring 50 nodes

In [70]:
envname = "BlockedMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.l1_norm
astar_l1_sol_gs, astar_l1_time_gs, astar_l1_memory_gs = astar(environment, astar_graph_search, heuristic_function)
greedy_l1_sol_ts, greedy_l1_time_ts, greedy_l1_memory_ts = greedy(environment, greedy_tree_search, heuristic_function)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 5                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

    
print("Tree search:")
print("\tSolution with Greedy Best Search l1_norm", greedy_l1_sol_ts)
print("\tNodes explored with Greedy Best Search l1_norm", greedy_l1_time_ts)

print()

print("\nGraph search:")
print("\tSolution with A* l1_norm", astar_l1_sol_gs)
print("\tNodes explored with A* l1_norm", astar_l1_time_gs)

########################################################
####### QUESTION 5                               #######
########################################################
Tree search:
	Solution with Greedy Best Search l1_norm time-out
	Nodes explored with Greedy Best Search l1_norm 10001


Graph search:
	Solution with A* l1_norm None
	Nodes explored with A* l1_norm 61


## Question 6
Consider the environment called BlockedMaze-v0:
```python
[
  ['C', 'C', 'S', 'C'],
  ['C', 'C', 'W', 'C'],
  ['C', 'C', 'C', 'C'],
  ['C', 'C', 'W', 'W'],
  ['C', 'C', 'W', 'G'],
]
startstate = 2
goalstate = 19

# You can create this environment with the following commands
envname = "BlockedMaze-v0"
environment = gym.make(envname)
```
### Which one of the following answers is correct?
- [ ] a. A* Graph Search with *Chebyshev* heuristic keeps a maximum of 17 nodes in memory
- [x] b. A* Graph Search with *l2_norm* heuristic keeps 15 nodes as with *Chebyshev* heuristic
- [ ] c. Greedy Best First Graph Search with *l1_norm* heuristic keeps 14 nodes

In [74]:
envname = "BlockedMaze-v0"
environment = gym.make(envname)

heuristic_function = Heu.l1_norm
_, astar_l1_time_gs, astar_l1_memory_gs = astar(environment, astar_graph_search, heuristic_function)
_, greedy_l1_time_gs, greedy_l1_memory_gs = greedy(environment, greedy_graph_search, heuristic_function)

heuristic_function = Heu.chebyshev
_, astar_ch_time_gs, astar_ch_memory_gs = astar(environment, astar_graph_search, heuristic_function)


print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 6                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

    
print("A*:")
print("\tNodes memorized with Chebyshev", astar_ch_memory_gs)
print("\tNodes memorized with l1_norm", astar_l1_memory_gs)

print("\nGreedy Best First:")
print("\tNodes memorized with l1_norm", greedy_l1_time_gs)

########################################################
####### QUESTION 6                               #######
########################################################
A*:
	Nodes memorized with Chebyshev 15
	Nodes memorized with l1_norm 15

Greedy Best First:
	Nodes memorized with l1_norm 61
